In [1]:
from collections.abc import Mapping
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm

import zipfile
import pandas as pd
import json
import openai

In [ ]:
client = Elasticsearch(IP)

In [2]:
df = pd.read_csv('./movies.csv', encoding='utf-8')

In [3]:
df = df[df['overview'].isna()!=True]
df = df.sort_values(by='averageRating', ascending=False)[:10000]
df = df.reset_index(drop=True)

# 새로운 열을 생성합니다
df['OVERVIEW_VECTOR'] = None

In [4]:
df

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,query,poster_path,overview,imdb_path,OVERVIEW_VECTOR
0,tt27051027,movie,Love Li,Love li,2024,168,"['Drama', 'Romance']",9.5,12678,https://api.themoviedb.org/3/find/tt27051027?e...,https://image.tmdb.org/t/p/w500//3jAvEJtFVSOaA...,An orphan grows up to become a gangster who fi...,https://www.imdb.com/title/tt27051027,None
1,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,1994,142,['Drama'],9.3,2919274,https://api.themoviedb.org/3/find/tt0111161?ex...,https://image.tmdb.org/t/p/w500//9cqNxx0GxF0bf...,Imprisoned in the 1940s for the double murder ...,https://www.imdb.com/title/tt0111161,None
2,tt0259534,movie,Ramayana: The Legend of Prince Rama,Ramayana: The Legend of Prince Rama,1993,135,"['Action', 'Adventure', 'Animation']",9.2,14992,https://api.themoviedb.org/3/find/tt0259534?ex...,https://image.tmdb.org/t/p/w500//6bGVpojv1f3ct...,"In Ayodhya, the royal palace of Kosala Kingdom...",https://www.imdb.com/title/tt0259534,None
3,tt16747572,movie,The Silence of Swastika,The Silence of Swastika,2021,56,"['Documentary', 'History']",9.2,10546,https://api.themoviedb.org/3/find/tt16747572?e...,https://image.tmdb.org/t/p/w500//j5aoyTXwDbPvS...,A documentary that distances the roots of Hitl...,https://www.imdb.com/title/tt16747572,None
4,tt0468569,movie,The Dark Knight,The Dark Knight,2008,152,"['Action', 'Crime', 'Drama']",9.0,2900164,https://api.themoviedb.org/3/find/tt0468569?ex...,https://image.tmdb.org/t/p/w500//qJ2tW6WMUDux9...,Batman raises the stakes in his war on crime. ...,https://www.imdb.com/title/tt0468569,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt1386925,movie,The Dead,The Dead,2010,105,"['Action', 'Adventure', 'Drama']",5.7,9682,https://api.themoviedb.org/3/find/tt1386925?ex...,https://image.tmdb.org/t/p/w500//1fyV05g2xPmnW...,When the last evacuation flight out of war-tor...,https://www.imdb.com/title/tt1386925,None
9996,tt13603966,movie,Heart of Stone,Heart of Stone,2023,122,"['Action', 'Crime', 'Thriller']",5.7,60940,https://api.themoviedb.org/3/find/tt13603966?e...,https://image.tmdb.org/t/p/w500//vB8o2p4ETnrfi...,An intelligence operative for a shadowy global...,https://www.imdb.com/title/tt13603966,None
9997,tt0430779,movie,Everyone's Hero,Everyone's Hero,2006,87,"['Adventure', 'Animation', 'Comedy']",5.7,9741,https://api.themoviedb.org/3/find/tt0430779?ex...,https://image.tmdb.org/t/p/w500//qFHXmKtS4GBoo...,A boy begins a grand journey to return Babe Ru...,https://www.imdb.com/title/tt0430779,None
9998,tt26547864,movie,The Conference,Konferensen,2023,100,"['Comedy', 'Horror', 'Mystery']",5.7,12502,https://api.themoviedb.org/3/find/tt26547864?e...,https://image.tmdb.org/t/p/w500//uLH5rUkRNwAUt...,A team-building conference for municipal emplo...,https://www.imdb.com/title/tt26547864,None


In [ ]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region)

def getEmbedding(text):
    request_body = json.dumps({
        "texts": [text],
        "input_type": "search_document",
        "truncate": "NONE"
    })

    # Bedrock API 호출
    response = bedrock.invoke_model(
        modelId="cohere.embed-multilingual-v3",
        body=request_body
    )

    # 응답 파싱
    response_body = json.loads(response['body'].read())
    embeddings = response_body['embeddings'][0]

    return embeddings

def getQueyrEmbedding(query):
    request_body = json.dumps({
        "texts": [query],
        "input_type": "search_query",
        "truncate": "NONE"
    })

    # Bedrock API 호출
    response = bedrock.invoke_model(
        modelId="cohere.embed-multilingual-v3",
        body=request_body
    )

    # 응답 파싱
    response_body = json.loads(response['body'].read())
    embeddings = response_body['embeddings'][0]

    return embeddings

In [ ]:
import pandas as pd
import numpy as np

# 진행 상황을 추적하기 위한 카운터
processed_count = 0

for idx, row in tqdm(df.iterrows()):
    overview = row['overview']
    
    overview_vector = getEmbedding(overview)

    # 벡터를 리스트로 변환하여 DataFrame에 저장
    df.at[idx, 'OVERVIEW_VECTOR'] = overview_vector
    
    # 진행 상황을 출력 (선택사항)
    processed_count += 1
    if processed_count % 100 == 0:
        print(f"Processed {processed_count} rows")

print(df.head())

In [ ]:
df.to_csv('df_movie.csv')

In [ ]:
index_mapping = {
    "mappings": {
        "properties": {
            "title_vector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine"
            },
            "content_vector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine"
            },
            "text": {"type": "text"},
            "title": {"type": "text"},
            "img_url": {"type": "keyword"},
            "vector_id": {"type": "long"}
        }
    }
}

client.indices.create(index="movie_vector_index", body=index_mapping)

In [ ]:
def dataframe_to_bulk_actions(df):
    for index, row in df.iterrows():
        yield {
            "_index": 'movie_vector_index',
            "_id": row['tconst'],
            "_source": {
                'url' : row["poster_path"],
                'title' : row["primaryTitle"],
                'text' : row["overview"],
                'overview_vector' : json.loads(row["OVERVIEW_VECTOR"]),
                'vector_id' : index
            }
        }

In [ ]:
start = 0
end = len(df)
batch_size = 100
for batch_start in tqdm(range(start, end, batch_size)):
    batch_end = min(batch_start + batch_size, end)
    batch_dataframe = df.iloc[batch_start:batch_end]
    actions = dataframe_to_bulk_actions(batch_dataframe)
    helpers.bulk(client, actions)

In [ ]:
diary = "내일은 중요한 면접이 있는 날이다. 잘 할 수 있을지 걱정된다. 준비는 열심히 했지만, 자꾸 불안한 마음이 든다. 오늘 밤 잠을 잘 수 있을지 모르겠다. 긍정적인 마인드를 가지려 노력해야겠다."
diary = "오늘 직장에서 너무 화가 났다. 내가 열심히 준비한 프로젝트를 동료가 자기 것인 양 발표했다. 나의 노력을 인정받지 못하는 것 같아 너무 분하고 억울하다. 이런 부당한 일이 또 일어나지 않도록 해야겠다"
query = f"당신은 심리 분석가처럼 행동합니다.\n또한 당신은 모든 영화에 대한 해박한 지식을 가지고 있습니다.\n아래의 일기를 읽어보고 글쓴이에게 위로가 될만한, 혹은 도움이 될만한 힐링 영화를 추천해주시기 바랍니다.\n\n### 일기 ###\n{diary}"

In [ ]:
question_embedding = getQueyrEmbedding(query)

In [ ]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        score = hit['_score']
        title = hit['_source']['title']
        text = hit['_source']['text']
        pretty_output = (f"\nID: {id}\nTitle: {title}\nSummary: {text}\nScore: {score}")
        print(pretty_output)

In [ ]:
query = {
    "knn": {
        "query_vector": question_embedding,
        "k": 3,
        "field": "overview_vector"  # This remains the same, as it refers to the field in your index
    },
    "_source": ["text", "title", "url"],
    "size": 3
}

response = client.search(
    index="movie_vector_index",
    body=query
)

pretty_response(response)

top_hit_summary = response['hits']['hits'][0]['_source']['text']

for row in response['hits']['hits']:
    row['_source']['traslated_text'] = getTraslation(row['_source']['text'])
[row['_source'] for row in response['hits']['hits']]